# Data Cleaning

This code analyzes the daily YouTube trending videos data sets available from: https://www.kaggle.com/datasets/datasnaek/youtube-new

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import re   #RE = regular expression https://docs.python.org/3/library/re.html  -> the functions in this module let you check if a particular string matches a given regular expression -> Used to perform string analysis
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns  #It is a Python data visualization library based on matplotlib and provides high-level interface for drawing attractive and informative statistical graphics (https://seaborn.pydata.org/ )
from matplotlib import cm  #cm stands for colormap
from datetime import datetime
import glob   #is a module that finds pathnames matching a specified pattern according to the rules of the Unix shell (e.g. the *)
import os
import json
import pickle #it is a module that implements binary protocols for serializing and de-serializing a Python object structure. “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation
#import six
sns.set()
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.options.mode.chained_assignment=None  #Ignores warning coming out when working on slices of dataframes

## Import and Read all the CSV files

In [2]:
AllCSV = [i for i in glob.glob('./Data/*.{}'.format('csv'))]

all_dataframes = [] #list to store each data frame separately
for csv in AllCSV:
    print(csv)
    df = pd.read_csv(csv, encoding='latin-1') #Had to add , encoding='latin-1' otherwise it was giving me error
    df['country'] = csv[7:9] # adding column 'country' with the prefix of the filename so to identify each dataset uniquely
    all_dataframes.append(df)

#Let's check one of the dataframes:
all_dataframes[0].head()

./Data/MXvideos.csv
./Data/INvideos.csv
./Data/DEvideos.csv
./Data/JPvideos.csv
./Data/KRvideos.csv
./Data/CAvideos.csv
./Data/RUvideos.csv
./Data/FRvideos.csv
./Data/USvideos.csv
./Data/GBvideos.csv


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,SbOwzAl9ZfQ,17.14.11,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,2017-11-13T06:06:22.000Z,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg,False,False,False,Disfruta la presencia del Chef Torreblanca en ...,MX
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,2017-11-13T05:11:58.000Z,La Voz Mexico 7,104972,271,174,369,https://i.ytimg.com/vi/klOV6Xh-DnI/default.jpg,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,MX
2,6L2ZF7Qzsbk,17.14.11,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,2017-11-13T17:00:02.000Z,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,https://i.ytimg.com/vi/6L2ZF7Qzsbk/default.jpg,False,False,False,La canciÃ³n del principio se llama âEste esp...,MX
3,hcY52MFWMDM,17.14.11,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,2017-11-13T03:47:10.000Z,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,https://i.ytimg.com/vi/hcY52MFWMDM/default.jpg,False,False,False,El video es de un Walmart en el pais centroame...,MX
4,_OXDcGPVAa4,17.14.11,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,2017-11-13T19:17:48.000Z,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,https://i.ytimg.com/vi/_OXDcGPVAa4/default.jpg,False,False,False,MI HERMANO NARRA MI RUTINA DE MAQUILLAJE\nhttp...,MX


## Data cleaning

### Convert strings to object types and boolean columns to categorical data

In [3]:
#Let's define the columns we want to convert to string:
string_cols = ['video_id','title','channel_title','tags','thumbnail_link','description','country']

#Let's define the columns we want to convert to categorical
categorical_cols = [column_name for column_name in all_dataframes[0].columns if all_dataframes[0][column_name].dtype=='bool']

In [4]:
for df in all_dataframes:
    for name in string_cols:
        df[name] = df[name].astype('str')
    for name in categorical_cols:
        df[name] = df[name].astype('category')

### Change dates format

The next step makes dealing with dates easier.

I convert the date columns (`trending date`, `publish time`) into proper datetime objects and I split the latter in two columns: `publish_date` and `publish_time`.

In [5]:
#Let's define the columns containing dates
date_cols = ['trending_date','publish_time']

for df in all_dataframes:
    for name in date_cols:
        if name=='trending_date':
            df[name] = pd.to_datetime(df[name],format='%y.%d.%m')
        else:
            df[name] = pd.to_datetime(df[name],format='%Y-%m-%dT%H:%M:%S.%fZ')
            #Let's splite the day and the timestamp
            df['publish_date'] = df[name].dt.normalize() #the normalize() is required t keep the dtype as datetime64, if you do not care use dt.date
            df['publish_timestamp'] =  df[name].dt.time

### Set video_id column as index

In [6]:
for df in all_dataframes:
    df.set_index('video_id', inplace=True)

and let's check the result:

In [7]:
all_dataframes[0].head()

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,publish_date,publish_timestamp
video_id,,,,,,,,,,,,,,,,,,
SbOwzAl9ZfQ,2017-11-14,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,2017-11-13 06:06:22,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg,False,False,False,Disfruta la presencia del Chef Torreblanca en ...,MX,2017-11-13,06:06:22
klOV6Xh-DnI,2017-11-14,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,2017-11-13 05:11:58,La Voz Mexico 7,104972,271,174,369,https://i.ytimg.com/vi/klOV6Xh-DnI/default.jpg,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,MX,2017-11-13,05:11:58
6L2ZF7Qzsbk,2017-11-14,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,2017-11-13 17:00:02,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,https://i.ytimg.com/vi/6L2ZF7Qzsbk/default.jpg,False,False,False,La canciÃ³n del principio se llama âEste esp...,MX,2017-11-13,17:00:02
hcY52MFWMDM,2017-11-14,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,2017-11-13 03:47:10,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,https://i.ytimg.com/vi/hcY52MFWMDM/default.jpg,False,False,False,El video es de un Walmart en el pais centroame...,MX,2017-11-13,03:47:10
_OXDcGPVAa4,2017-11-14,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,2017-11-13 19:17:48,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,https://i.ytimg.com/vi/_OXDcGPVAa4/default.jpg,False,False,False,MI HERMANO NARRA MI RUTINA DE MAQUILLAJE\nhttp...,MX,2017-11-13,19:17:48


### Check for missing values

In [8]:
nnull=0

for df in all_dataframes:
    nnull+=df.isnull().sum().sum()  #The first sum sums all null values in a column, while second sum() sums for all columns

print('Number of missing values: ', nnull)

Number of missing values:  0


This tells me that the dataset contain no missing values

### Combine all datasets in a unique large dataset

In [9]:
combined_df = pd.concat(all_dataframes)

I now refine the combined dataframe by sorting the entries of the dataset by `trending_date` (latest trending videos on top) and dropping eventual duplicates.

For safety I first create a copy of the dataframe

In [10]:
# Copy of original df
backup_df = combined_df

In [11]:
combined_df = combined_df.reset_index().sort_values('trending_date',ascending=False).drop_duplicates('video_id',keep='first').set_index('video_id')

In [12]:
# Printing results
display(combined_df[['publish_date','publish_timestamp','trending_date', 'country']].head())
print('Number of entries: ', len(combined_df))

,publish_date,publish_timestamp,trending_date,country
video_id,,,,
4YFo4bdMO8Q,2018-05-11,04:06:35,2018-06-14,GB
L_cxNGfEd_0,2018-06-13,10:54:55,2018-06-14,FR
ENVlcPpTuh4,2018-06-13,14:56:22,2018-06-14,KR
5BeWvP521s8,2018-06-12,08:00:02,2018-06-14,KR
nOUlB0i2MaA,2018-06-13,19:10:31,2018-06-14,KR


Number of entries:  184287


I also do the same (sort + remove duplicates) for the individual dataframes:

In [13]:
for df in all_dataframes:
    df = df.reset_index().sort_values('trending_date', ascending=False).set_index('video_id')

### Explore and combine information from json files

Together with csv data file come also JSON files.
I read one to check what type of information it cointains, and then I read another one at random to see if they contain the same or different data.

In [14]:
#read first JSON file
with open('./Data/US_category_id.json','r') as f:
    data = f.read()
#parse file
obj = json.loads(data)
#print
obj

{'kind': 'youtube#videoCategoryListResponse',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
   'id': '1',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Film & Animation',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
   'id': '2',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Autos & Vehicles',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
   'id': '10',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Music',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"',
   'id': '15',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdnt

In [15]:
#read JSON file
with open('./Data/FR_category_id.json','r') as f2:
    data2 = f2.read()
#parse file
obj2 = json.loads(data2)
#print
obj2

{'kind': 'youtube#videoCategoryListResponse',
 'etag': '"ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2qTj13hkQZk"',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': '"ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
   'id': '1',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Film & Animation',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"ld9biNPKjAjgjV7EZ4EKeEGrhao/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
   'id': '2',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Autos & Vehicles',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"ld9biNPKjAjgjV7EZ4EKeEGrhao/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
   'id': '10',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Music',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"ld9biNPKjAjgjV7EZ4EKeEGrhao/HwXKamM1Q20q9BN-oBJavSGkfDI"',
   'id': '15',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdnt

Both JSON files are very similar. 

Each JSON file contains `id` ranging from 1 to 44 (both inclusive). And with each id there a category related (e.g. 10 = Music). Hence, we can use any one of the JSON files to map category to category id in our dataframe.

There are some missing IDs in all the files, so for those we will assign a 'NaN' category.

I define a few functions to do this job:

In [16]:
def CheckMissingIDs(category_id):
    missing_id = []
    for i in range(1,45):
        if str(i) not in category_id.keys():
            missing_id.append(i)
    return missing_id
        
def ExtractCategory():
    category_id = {}
    with open('./Data/DE_category_id.json','r') as f:
        d = json.load(f)
        for category in d['items']:
            category_id[category['id']] = category['snippet']['title']
    missing_id = CheckMissingIDs(category_id)
    for id in missing_id:
        category_id[str(id)] = 'None'
    return category_id

print(ExtractCategory())

def SetCategory(id):
    category_id = ExtractCategory()
    return category_id[str(id)]

{'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '30': 'Movies', '31': 'Anime/Animation', '32': 'Action/Adventure', '33': 'Classics', '34': 'Comedy', '35': 'Documentary', '36': 'Drama', '37': 'Family', '38': 'Foreign', '39': 'Horror', '40': 'Sci-Fi/Fantasy', '41': 'Thriller', '42': 'Shorts', '43': 'Shows', '44': 'Trailers', '3': 'None', '4': 'None', '5': 'None', '6': 'None', '7': 'None', '8': 'None', '9': 'None', '11': 'None', '12': 'None', '13': 'None', '14': 'None', '16': 'None', '29': 'None'}


I now add the information to the combined data frame

In [17]:
combined_df['category'] = combined_df['category_id'].map(SetCategory)
combined_df.head() 

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,publish_date,publish_timestamp,category
video_id,,,,,,,,,,,,,,,,,,,
4YFo4bdMO8Q,2018-06-14,KYLE - Ikuyo feat. 2 Chainz & Sophia Black [A...,SuperDuperKyle,10,2018-05-11 04:06:35,"Kyle|""SuperDuperKyle""|""Ikuyo""|""2 Chainz""|""Soph...",607552,18271,274,1423,https://i.ytimg.com/vi/4YFo4bdMO8Q/default.jpg,False,False,False,Debut album 'Light of Mine' out now: http://ky...,GB,2018-05-11,04:06:35,Music
L_cxNGfEd_0,2018-06-14,CLASSE PAS CLASSE - PAROLE DE CHAT,Parole de chat,15,2018-06-13 10:54:55,"classe|""pas classe""|""parole""|""parole de chat""|...",186468,28742,233,2168,https://i.ytimg.com/vi/L_cxNGfEd_0/default.jpg,False,False,False,"Classe pas classe, hommage aux Robins des bois...",FR,2018-06-13,10:54:55,Pets & Animals
ENVlcPpTuh4,2018-06-14,"[ì í2018, íì¸í¡í¡] ìë¬´ë¦¬ ì§ë¬¸ì´...",MBCNEWS,25,2018-06-13 14:56:22,"MBC|""MBCë´ì¤""|""ë´ì¤ë°ì¤í¬""|""newsdesk""|""...",836427,2254,801,3551,https://i.ytimg.com/vi/ENVlcPpTuh4/default.jpg,False,False,False,"ì í2018, 6.13ì§ë°©ì ê±°, ê°íë°©ì¡, ì...",KR,2018-06-13,14:56:22,News & Politics
5BeWvP521s8,2018-06-14,The genetic test result for the Dog Mother-Cat...,SBS TVëë¬¼ëì¥xì ëë©ë´,15,2018-06-12 08:00:02,"ì ëë©ë´|""ëë¬¼ëì¥""|""TVëë¬¼ëì¥""|""...",656898,5222,225,821,https://i.ytimg.com/vi/5BeWvP521s8/default.jpg,False,False,False,The unbelievable result baffled the entire nei...,KR,2018-06-12,08:00:02,Pets & Animals
nOUlB0i2MaA,2018-06-14,2018 êµ­ë¯¼ì ì í - ì ê±°ë°©ì¡ 'ìì ê...,SBS ë´ì¤,25,2018-06-13 19:10:31,"ë´ì¤|""NEWS""|""news""|""SBS""|""sbs""|""SBS NEWS""|""S...",433188,0,0,175,https://i.ytimg.com/vi/nOUlB0i2MaA/default.jpg,False,True,False,'ì ê±°ë°©ì¡ì ëª ê°' SBSê° ì¨ë¤! SBSê°...,KR,2018-06-13,19:10:31,News & Politics


I repeat the same procedure also for the single dataframes:

In [18]:
for df in all_dataframes:
    df['category'] = df['category_id'].map(SetCategory)
    
# Verify results for one of the dataframes
all_dataframes[0].head()

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,publish_date,publish_timestamp,category
video_id,,,,,,,,,,,,,,,,,,,
SbOwzAl9ZfQ,2017-11-14,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,2017-11-13 06:06:22,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg,False,False,False,Disfruta la presencia del Chef Torreblanca en ...,MX,2017-11-13,06:06:22,Entertainment
klOV6Xh-DnI,2017-11-14,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,2017-11-13 05:11:58,La Voz Mexico 7,104972,271,174,369,https://i.ytimg.com/vi/klOV6Xh-DnI/default.jpg,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,MX,2017-11-13,05:11:58,People & Blogs
6L2ZF7Qzsbk,2017-11-14,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,2017-11-13 17:00:02,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,https://i.ytimg.com/vi/6L2ZF7Qzsbk/default.jpg,False,False,False,La canciÃ³n del principio se llama âEste esp...,MX,2017-11-13,17:00:02,News & Politics
hcY52MFWMDM,2017-11-14,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,2017-11-13 03:47:10,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,https://i.ytimg.com/vi/hcY52MFWMDM/default.jpg,False,False,False,El video es de un Walmart en el pais centroame...,MX,2017-11-13,03:47:10,News & Politics
_OXDcGPVAa4,2017-11-14,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,2017-11-13 19:17:48,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,https://i.ytimg.com/vi/_OXDcGPVAa4/default.jpg,False,False,False,MI HERMANO NARRA MI RUTINA DE MAQUILLAJE\nhttp...,MX,2017-11-13,19:17:48,Howto & Style


### Save cleaned dataframes

I am now going to save these newly created cleaned dataframes in a pickle format so that it preserves the state of the dataframe.

In [19]:
os.mkdir('./Data_cleaned/')

In [20]:
combined_df.to_pickle('./Data_cleaned/CombinedVideos.pkl')
for df in all_dataframes:
    country_tag = df['country'][0]
    df.to_pickle('./Data_cleaned/'+country_tag+'videos.pkl')    